# Técnica Record Linkage no Python

Vamos implementar um código que utilize a técnica de Record Linkage, aplicando a Lógica Difusa (Fuzzy Logic) para comparar informações de dois conjuntos de dados sobre livros.

Utilizaremos o algoritmo de Levenshtein, amplamente conhecido e utilizado para esta tarefa. Este algoritmo permite comparar duas strings e retornar um valor de similaridade entre 0 e 1, onde 0 indica nenhuma correspondência entre os caracteres nas mesmas posições e 1 representa uma correspondência perfeita.


## 1. Primeira Tentativa

Na nossa primeira tentativa de implementar um código que utilize a técnica de Record Linkage, vamos comparar apenas os títulos dos livros.

Note que aqui nem sempre adotaremos as melhores práticas no código, pois o objetivo é ser didático.

### 1.1 Carregar Conjuntos de Dados
Vamos começar carregando os conjuntos de dados que contêm informações sobre livros. Vamos usar a biblioteca `pandas` para ler os arquivos CSV e visualizar os primeiros registros de cada DataFrame.

In [5]:
import recordlinkage as rl
import pandas as pd

# Carregar os conjuntos de dados
dfA = pd.read_csv('book_release_year.csv', delimiter = ';', encoding = 'UTF-8', index_col = 'Índice')
dfB = pd.read_csv('book_sold_copies.csv', delimiter = ';', encoding = 'UTF-8', index_col = 'Índice')

# Exibir os primeiros registros dos DataFrames
display(dfA.head())
display(dfB.head())

,Título,Autor,Ano Publicação
Índice,,,
A1,Dune,Frank Herbert,1965
A2,Foundation,Isaac Asimov,1951
A3,Foundation and Earth,Isaac Asimov,1986
A4,Foundation and Empire,Isaac Asimov,1952
A5,Foundations Edge,Isaac Asimov,1982


,Título,Autor,Tiragem
Índice,,,
B1,The Two Towers,J.R.R. Tolkien,150.000.000
B2,The Fellowship of the Ring,J.R.R. Tolkien,150.000.000
B3,The Return of the King,J.R.R. Tolkien,150.000.000
B4,Dune,Frank Herbert,20.000.000
B5,The Hitchhiker's Guide to the Galaxy,Douglas Adams,14.000.000


### 1.2 Criação do Indexador
Para realizar a ligação de registros, vamos criar um objeto `Index` da biblioteca `recordlinkage`. Esse objeto nos permite definir como os registros dos dois conjuntos de dados serão comparados. Usaremos o método `full` para comparar todos os registros.

In [6]:
indexador_1 = rl.Index()  # Cria o objeto indexador
indexador_1.full()  # Define que todos os elementos do dfA e dfB serão comparados um a um
candidatos_1 = indexador_1.index(dfA, dfB)  # Tabela que define cada elemento do dfA e dfB que serão comparados

# Exibir os candidatos
display(candidatos_1)


MultiIndex([( 'A1',  'B1'),
            ( 'A1',  'B2'),
            ( 'A1',  'B3'),
            ( 'A1',  'B4'),
            ( 'A1',  'B5'),
            ( 'A1',  'B6'),
            ( 'A1',  'B7'),
            ( 'A1',  'B8'),
            ( 'A1',  'B9'),
            ( 'A1', 'B10'),
            ...
            ('A14',  'B5'),
            ('A14',  'B6'),
            ('A14',  'B7'),
            ('A14',  'B8'),
            ('A14',  'B9'),
            ('A14', 'B10'),
            ('A14', 'B11'),
            ('A14', 'B12'),
            ('A14', 'B13'),
            ('A14', 'B14')],
           names=['Índice_1', 'Índice_2'], length=196)

### 1.3 Criação do Objeto Comparar
Em seguida, criaremos um objeto `Compare` para definir os critérios de comparação entre os registros dos dois conjuntos de dados. Usaremos o algoritmo de Levenshtein para comparar os títulos dos livros.



In [7]:
comparador_1 = rl.Compare()

# Comparação entre strings: Título
comparador_1.string("Título",  # Coluna do 1º dataframe que será comparada
                    "Título",  # Coluna do 2º dataframe para comparar
                    method="levenshtein",  # Algoritmo de comparação de similaridade entre strings
                    threshold=0.8,  # Limiar de corte, ou nível de similaridade entre as strings
                    label="Título")  # Rótulo da comparação

# Executa as comparações
resultados = comparador_1.compute(candidatos_1,  # Tabela que define cada elemento do dfA e dfB que serão comparados
                                  dfA, dfB)

# Exibir os resultados
display(resultados)


Título
Índice_1 Índice_2        
A1       B1           0.0
         B2           0.0
         B3           0.0
         B4           1.0
         B5           0.0
...                   ...
A14      B10          0.0
         B11          0.0
         B12          1.0
         B13          0.0
         B14          1.0

[196 rows x 1 columns]

### 1.4 Selecionar os Resultados
Após a comparação, podemos querer limpar o DataFrame resultante removendo colunas indesejadas. Aqui, especificamos as colunas a serem removidas e filtramos o DataFrame.


In [8]:
tabela_associação_1 = resultados[resultados['Título'] > 0]  # Filtra do df_resultados as comparações que foram igual ou superior ao limiar definido

tabela_associação_1.reset_index(inplace=True)  # Reseta o índice do df, transformando os índices do dfA e dfB em colunas
tabela_associação_1 = tabela_associação_1[['Índice_1', 'Índice_2']]  # Mantém apenas os índices

# Juntar os DataFrames baseados nos índices
dfs_unidos = tabela_associação_1.merge(dfA, left_on='Índice_1', right_on='Índice', how='left')
dfs_unidos = dfs_unidos.merge(dfB, left_on='Índice_2', right_on='Índice', how='left')

# Exibir o DataFrame resultante
display(dfs_unidos)


,Índice_1,Índice_2,Título_x,Autor_x,Ano Publicação,Título_y,Autor_y,Tiragem
0,A1,B4,Dune,Frank Herbert,1965,Dune,Frank Herbert,20.000.000
1,A2,B7,Foundation,Isaac Asimov,1951,Foundation,Isaac Asimov,4.000.000
2,A3,B11,Foundation and Earth,Isaac Asimov,1986,Foundation and Earth,Isaac Asimov,4.000.000
3,A4,B8,Foundation and Empire,Isaac Asimov,1952,Foundation and Empire,Isaac Asimov,4.000.000
4,A5,B10,Foundations Edge,Isaac Asimov,1982,Foundation's Edge,Isaac Asimov,4.000.000
5,A6,B13,Frankenstein,Mary Wollstonecraft Shelley,1818,Frankenstein,Mary W. Shelley,1.000.000
6,A7,B6,Neuromancer,William Gibson,1984,Neuromancer,William Gibson,6.500.000
7,A8,B9,Second Foundation,Isaac Asimov,1953,Second Foundation,Isaac Asimov,4.000.000
8,A9,B2,The Fellowship of the Ring,John Ronald Reuel Tolkien,1954,The Fellowship of the Ring,J.R.R. Tolkien,150.000.000
9,A10,B5,The Hitchhikers Guide to the Galaxy,Douglas Adams,1979,The Hitchhiker's Guide to the Galaxy,Douglas Adams,14.000.000


### 1.5 Análise dos Resultados
Observe que a ausência do apóstrofo (') nos títulos do conjunto B não impediu a correta associação das obras _"The Hitchhiker's Guide to the Galaxy"_ e _"Foundation's Edge"_. No entanto, a obra _"The War of the Words"_ de Rachael Jolley foi erroneamente associada a _"The War of the Worlds"_ de H. G. Wells. Nesse caso, a diferença na letra "l" foi significativa.


# 2. Segunda Tentativa

Desta vez, vamos implementar um algoritmo mais sofisticado, que vai comparar não apenas o título, mas também o último nome do autor. Para isso, antes da implementação, devemos criar uma nova coluna nos dois conjuntos de dados com o último nome do autor. Note que no conjunto B, o nome dos autores está abreviado, o que pode ser um problema. Também há um pequeno erro de digitação no nome de _Frank Hebert_, mas podemos lidar com isso.


### 2.1. Extrair Último Nome do Autor
Vamos criar uma função que quebra a string do nome e retorna o último nome. Em seguida, usaremos o método `apply` do pandas para criar uma nova coluna com o novo atributo.

In [9]:
# Função que retorna o último nome do autor
def ultimo_nome(nome):
    lista_nomes = nome.split()
    ultimo_nome = lista_nomes[-1]
    return ultimo_nome

# Aplicar a função na coluna 'Autor', devolvendo o resultado na coluna 'Último nome do autor'
dfA['Último nome do autor'] = dfA['Autor'].apply(ultimo_nome)
dfB['Último nome do autor'] = dfB['Autor'].apply(ultimo_nome)

# Exibir os DataFrames atualizados
display(dfA.head())
display(dfB.head())


,Título,Autor,Ano Publicação,Último nome do autor
Índice,,,,
A1,Dune,Frank Herbert,1965,Herbert
A2,Foundation,Isaac Asimov,1951,Asimov
A3,Foundation and Earth,Isaac Asimov,1986,Asimov
A4,Foundation and Empire,Isaac Asimov,1952,Asimov
A5,Foundations Edge,Isaac Asimov,1982,Asimov


,Título,Autor,Tiragem,Último nome do autor
Índice,,,,
B1,The Two Towers,J.R.R. Tolkien,150.000.000,Tolkien
B2,The Fellowship of the Ring,J.R.R. Tolkien,150.000.000,Tolkien
B3,The Return of the King,J.R.R. Tolkien,150.000.000,Tolkien
B4,Dune,Frank Herbert,20.000.000,Herbert
B5,The Hitchhiker's Guide to the Galaxy,Douglas Adams,14.000.000,Adams


### 2.2. Indexador e Comparador
Novamente, vamos criar um indexador e comparador, mas dessa vez, o comparador irá verificar também o último nome do autor.

In [10]:
indexador_2 = rl.Index()  # Cria o objeto indexador
indexador_2.full()  # Seleciona o atributo criação dos candidatos
candidatos_2 = indexador_2.index(dfA, dfB)  # Cria a tabela de candidatos

comparador_2 = rl.Compare()

# Comparação entre strings: Título
comparador_2.string("Título", "Título", method="levenshtein", threshold=0.8, label="Título")
# Comparação entre strings: Último nome do autor
comparador_2.string("Último nome do autor", "Último nome do autor", method="levenshtein", threshold=0.8, label="Último nome do autor")

# Executa as comparações
resultados = comparador_2.compute(candidatos_2, dfA, dfB)

# Exibir os resultados
display(resultados)

Título  Último nome do autor
Índice_1 Índice_2                              
A1       B1           0.0                   0.0
         B2           0.0                   0.0
         B3           0.0                   0.0
         B4           1.0                   1.0
         B5           0.0                   0.0
...                   ...                   ...
A14      B10          0.0                   0.0
         B11          0.0                   0.0
         B12          1.0                   1.0
         B13          0.0                   0.0
         B14          1.0                   0.0

[196 rows x 2 columns]

### 2.3. Filtrar Resultados e Unir DataFrames
Apenas as linhas cuja a soma dos scores é igual a 2, ou seja, quando o `título` do livro e `último nome do autor` têm similaridade igual ou superior a 0.8, serão consideradas.

Vamos fazer a junção apenas com o que interessa de cada conjunto para ter um resultado mais limpo. Do conjunto, foi selecionado apenas os atributos `Título`, `Autor`, `Ano Publicação`, e do conjunto B apenas a `Tiragem`, para evitar redundância de atributos.

In [11]:
# Filtrar os resultados
tabela_associação_2 = resultados[resultados.sum(axis=1) == 2]

# Resetar o índice e manter apenas os índices
tabela_associação_2.reset_index(inplace=True)
tabela_associação_2 = tabela_associação_2[['Índice_1', 'Índice_2']]

# Juntar os DataFrames baseados nos índices
dfs_unidos = tabela_associação_2.merge(dfA[['Título', 'Autor', 'Ano Publicação']], left_on='Índice_1', right_on='Índice', how='left')
dfs_unidos = dfs_unidos.merge(dfB[['Tiragem']], left_on='Índice_2', right_on='Índice', how='left')

# Exibir o DataFrame resultante
display(dfs_unidos)


,Índice_1,Índice_2,Título,Autor,Ano Publicação,Tiragem
0,A1,B4,Dune,Frank Herbert,1965,20.000.000
1,A2,B7,Foundation,Isaac Asimov,1951,4.000.000
2,A3,B11,Foundation and Earth,Isaac Asimov,1986,4.000.000
3,A4,B8,Foundation and Empire,Isaac Asimov,1952,4.000.000
4,A5,B10,Foundations Edge,Isaac Asimov,1982,4.000.000
5,A6,B13,Frankenstein,Mary Wollstonecraft Shelley,1818,1.000.000
6,A7,B6,Neuromancer,William Gibson,1984,6.500.000
7,A8,B9,Second Foundation,Isaac Asimov,1953,4.000.000
8,A9,B2,The Fellowship of the Ring,John Ronald Reuel Tolkien,1954,150.000.000
9,A10,B5,The Hitchhikers Guide to the Galaxy,Douglas Adams,1979,14.000.000


### 2.3. Remover colunas indesejadas
Apenas as linhas cuja a soma dos scores é igual 2, ou seja, quando o `titulo` do livro e `ultimo nome do autor` tinha similaridade igual ou superior à 0.8, serão considerados.

In [13]:
remove = ['Índice_1','Índice_2']
dfs_unidos = dfs_unidos[[coluna for coluna in dfs_unidos.columns if coluna not in remove]]
display(dfs_unidos)

,Título,Autor,Ano Publicação,Tiragem
0,Dune,Frank Herbert,1965,20.000.000
1,Foundation,Isaac Asimov,1951,4.000.000
2,Foundation and Earth,Isaac Asimov,1986,4.000.000
3,Foundation and Empire,Isaac Asimov,1952,4.000.000
4,Foundations Edge,Isaac Asimov,1982,4.000.000
5,Frankenstein,Mary Wollstonecraft Shelley,1818,1.000.000
6,Neuromancer,William Gibson,1984,6.500.000
7,Second Foundation,Isaac Asimov,1953,4.000.000
8,The Fellowship of the Ring,John Ronald Reuel Tolkien,1954,150.000.000
9,The Hitchhikers Guide to the Galaxy,Douglas Adams,1979,14.000.000


### 2.4 Análise dos resultados

Finalmente, temos um conjunto com corretamente unido, independente dos similaridades entre obras ou erros de digitação.